### Avalie o desempenho de algoritmos de aprendizado de máquina com reamostragem

Você precisa saber o desempenho de seus algoritmos em dados não vistos. A melhor forma de avaliar o desempenho de um algoritmo seria fazer previsões para novos dados aos quais você já sabe as respostas. A segunda melhor maneira é usar técnicas inteligentes de estatísticas chamadas métodos de reamostragem que permitem fazer estimativas precisas de quão bem seu algoritmo executar em novos dados. Neste capítulo, você descobrirá como estimar a precisão de seus algoritmos de aprendizado de máquina usando métodos de reamostragem em Python e scikit-learn no Conjunto de dados dos índios Pima. Vamos começar.

#### Avalie algoritmos de aprendizado de máquina

Por que você não pode treinar seu algoritmo de aprendizado de máquina em seu conjunto de dados e usar previsões de esse mesmo conjunto de dados para avaliar algoritmos de aprendizado de máquina? A resposta simples é over tting.

Imagine um algoritmo que se lembra de cada observação que é mostrada durante o treinamento. Se você avaliou seu algoritmo de aprendizado de máquina no mesmo conjunto de dados usado para treinar o algoritmo e, em seguida, um algoritmo como esse teria uma pontuação perfeita no conjunto de dados de treinamento. Mas as previsões que feito em novos dados seria terrível. Devemos avaliar nossos algoritmos de aprendizado de máquina em dados que não são usados para treinar o algoritmo.

A avaliação é uma estimativa que podemos usar para falar sobre o quão bem pensamos o algoritmo pode realmente fazer na prática. Não é uma garantia de desempenho. Uma vez que estimamos o desempenho de nosso algoritmo, podemos treinar novamente o algoritmo final em todo o treinamento conjunto de dados e prepará-lo para uso operacional. Em seguida, vamos olhar para quatro diferentes técnicas que podemos usar para dividir nosso conjunto de dados de treinamento e criar estimativas úteis de desempenho para nossos algoritmos de aprendizado de máquina:

. Conjuntos de treinamento e teste.
. Validação cruzada k-fold.
. Deixe um de fora da validação cruzada.
. Divisões aleatórias repetidas do trem de teste.

#### Dividido em conjuntos de treinamento e teste

O método mais simples que podemos usar para avaliar o desempenho de um aprendizado de máquina algoritmo é usar diferentes conjuntos de dados de treinamento e teste. Podemos pegar nosso conjunto de dados original e dividi-lo em duas partes. Treine o algoritmo na primeira parte, faça previsões na segunda parte e avaliar as previsões contra os resultados esperados. O tamanho da divisão pode depender no tamanho e especificidades do seu conjunto de dados, embora seja comum usar 67% dos dados para treinamento e os 33% restantes para testes.

Esta técnica de avaliação de algoritmo é muito rápida. É ideal para grandes conjuntos de dados (milhões de registros) onde há fortes evidências de que ambas as divisões dos dados são representativas do Problema subjacente. Por causa da velocidade, é útil usar essa abordagem quando o algoritmo você está investigando é lento para treinar. Uma desvantagem dessa técnica é que ela pode ter um alto variância. Isso significa que diferenças no conjunto de dados de treinamento e teste podem resultar em diferenças na estimativa de precisão. No exemplo abaixo, dividimos o conjunto de dados dos índios Pima em divisões de 67%/33% para treinamento e teste e avaliar a precisão de uma regressão logística modelo.

In [4]:
# Evaluate using a train and a test set
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

filename = 'datasets/diabetes.csv'
names = ['preg', 'plas', 'pres',
         'skin', 'test', 'mass', 
         'pedi', 'age', 'class']

dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

test_size = 0.33
seed = 7

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,
                                                    random_state=seed)

model = LogisticRegression()
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
print(f"Accuracy: {result*100.0}")

Accuracy: 78.74015748031496


Podemos observar que a acurácia estimada para o modelo foi de aproximadamente 75%. Observe que além de especificar o tamanho da divisão, também especificamos a semente aleatória. Porque o divisão dos dados for aleatória, queremos garantir que os resultados sejam reprodutíveis. Ao especificar a semente aleatória, garantimos que obteremos os mesmos números aleatórios toda vez que executarmos o código e, por sua vez, a mesma divisão de dados. Isso é importante se quisermos comparar esse resultado com a precisão estimada de outro algoritmo de aprendizado de máquina ou o mesmo algoritmo com um configuração diferente. Para garantir que a comparação seja maçãs por maçãs, devemos garantir que eles são treinados e testados exatamente com os mesmos dados.

#### Validação Cruzada K-fold

A validação cruzada é uma abordagem que você pode usar para estimar o desempenho de uma máquina algoritmo de aprendizado com menos variância do que uma única divisão de conjunto de teste de trem. Funciona dividindo o conjunto de dados em k partes (por exemplo, k = 5 ou k = 10). Cada divisão dos dados é chamada de dobra. O o algoritmo é treinado em k 􀀀 1 dobras com uma retida e testada na dobra retida. Isso é repetido para que cada dobra do conjunto de dados tenha a chance de ser o conjunto de teste retido. Depois executando a validação cruzada, você acaba com k pontuações de desempenho diferentes que você pode resumir usando uma média e um desvio padrão.

O resultado é uma estimativa mais confiável do desempenho do algoritmo em novos dados. Isso é mais preciso porque o algoritmo é treinado e avaliado várias vezes em dados diferentes. A escolha de k deve permitir que o tamanho de cada partição de teste seja grande o suficiente para ser um valor razoável amostra do problema, permitindo repetições suficientes da avaliação do teste de treinamento do algoritmo para fornecer uma estimativa justa do desempenho do algoritmo em dados não vistos. Para modesto conjuntos de dados dimensionados em milhares ou dezenas de milhares de registros, valores k de 3, 5 e 10 são comum. No exemplo abaixo, usamos validação cruzada de 10 vezes.

In [ ]:
# Evaluate using Cross Validation
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

filename = 'datasets/diabetes.csv'
names = ['preg', 'plas', 'pres',
         'skin', 'test', 'mass', 
         'pedi', 'age', 'class']

dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

num_folds = 10
seed = 7

kfold = KFold(n_splits=num_folds, random_state=seed)
model = LogisticRegression()
results = cross_val_score(model, X, Y, cv=kfold)

print(f"Accuracy: {results.mean()*100.0%.3%}(%.3f%%)") % (, results.std()*100.0)

Você pode ver que relatamos a média e o desvio padrão do desempenho medir. Ao resumir as medidas de desempenho, é uma boa prática resumir o distribuição das medidas, neste caso assumindo uma distribuição gaussiana de desempenho (uma suposição muito razoável) e registrando a média e o desvio padrão.

#### Deixar um de fora da validação cruzada

Você pode configurar a validação cruzada para que o tamanho da dobra seja 1 (k é definido como o número de observações em seu conjunto de dados). Essa variação de validação cruzada é chamada de cruzamento leave-one-out validação. O resultado é um grande número de medidas de desempenho que podem ser resumidas em um esforço para fornecer uma estimativa mais razoável da precisão de seu modelo em dados não vistos. Uma desvantagem é que pode ser um procedimento computacionalmente mais caro do que o cruzamento k-fold validação. No exemplo abaixo, usamos a validação cruzada leave-one-out.

In [ ]:
# Evaluate using Leave One Out Cross Validation
from pandas import read_csv
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
num_folds = 10
loocv = LeaveOneOut()
model = LogisticRegression()
results = cross_val_score(model, X, Y, cv=loocv)
print("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0)

Você pode ver no desvio padrão que a pontuação tem mais variância do que o cruzamento k-fold resultados de validação descritos acima.

#### Divisões aleatórias repetidas do trem de teste

Outra variação na validação cruzada k-fold é criar uma divisão aleatória dos dados como o divisão de treinamento/teste descrita acima, mas repita o processo de divisão e avaliação do algoritmo várias vezes, como validação cruzada. Isso tem a velocidade de usar uma divisão de treinamento/teste e a redução na variância no desempenho estimado da validação cruzada k-fold. Você também pode repita o processo muitas vezes, conforme necessário, para melhorar a precisão. Um lado negativo é que as repetições podem incluir muitos dos mesmos dados no treino ou na divisão do teste de corrida para corrida, introduzir redundância na avaliação. O exemplo abaixo divide os dados em 67%/33% split train/test e repete o processo 10 vezes.

In [ ]:
# Evaluate using Shuffle Split Cross Validation
from pandas import read_csv
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
n_splits = 10
test_size = 0.33
seed = 7
kfold = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)
model = LogisticRegression()
results = cross_val_score(model, X, Y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0)

Podemos ver que, neste caso, a distribuição da medida de desempenho está em pé de igualdade com validação cruzada k-fold acima.

#### Quais técnicas usar quando

Esta seção lista algumas dicas para considerar qual técnica de reamostragem usar em diferentes circunstâncias. posturas. Geralmente a validação cruzada k-fold é o padrão-ouro para avaliar o desempenho de um algoritmo de aprendizado de máquina em dados não vistos com k definido como 3, 5 ou 10. Usar uma divisão de treinamento/teste é bom para velocidade ao usar um algoritmo lento e produz estimativas de desempenho com menor viés ao usar grandes conjuntos de dados. Técnicas como validação cruzada de deixar um de fora e divisões aleatórias repetidas podem ser úteis intermediários ao tentar equilibrar a variação no desempenho estimado, modelo velocidade de treinamento e tamanho do conjunto de dados. O melhor conselho é experimentar e encontrar uma técnica para o seu problema que seja rápida e produz estimativas razoáveis de desempenho que você pode usar para tomar decisões. Se em dúvida, usar validação cruzada de 10 vezes.

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

### End.